In [1]:
import mlflow
from mlflow.sklearn import log_model

modelo_final = mlflow.sklearn.load_model("data/06_models/modelo_vencedor")

with mlflow.start_run(run_name="FixModeloComArtefato"):
    log_model(
        sk_model=modelo_final,
        artifact_path="modelo_vencedor",
        registered_model_name="ModeloArremessoKobe"
    )

    print("✅ Modelo logado com sucesso com artefatos!")


✅ Modelo logado com sucesso com artefatos!


Registered model 'ModeloArremessoKobe' already exists. Creating a new version of this model...
Created version '2' of model 'ModeloArremessoKobe'.


In [3]:
from mlflow import MlflowClient

client = MlflowClient()
client.set_registered_model_alias("ModeloArremessoKobe", "prod", 2)  # Use o número da nova versão


In [4]:
import mlflow
from mlflow import MlflowClient

with mlflow.start_run(run_name="ReRegistroModelo"):
    model_uri = mlflow.get_artifact_uri("modelo_vencedor")
    result = mlflow.register_model(model_uri, "ModeloArremessoKobe")

    print("✔️ Registrado:", result.name, "v", result.version)

    # Atribuir alias novamente
    client = MlflowClient()
    client.set_registered_model_alias("ModeloArremessoKobe", "prod", result.version)


Registered model 'ModeloArremessoKobe' already exists. Creating a new version of this model...
Created version '3' of model 'ModeloArremessoKobe'.


✔️ Registrado: ModeloArremessoKobe v 3


In [5]:
# 📌 Aplicacao_Modelo_Prod.ipynb

import mlflow
import mlflow.sklearn
from mlflow.sklearn import log_model
from mlflow import MlflowClient
from sklearn.metrics import f1_score, log_loss
from pathlib import Path
import pandas as pd

# ✅ 1. Configurar o tracking URI
mlflow.set_tracking_uri("file:///C:/Users/Cristina/Documents/myproject/mlruns")
print("[INFO] Tracking URI:", mlflow.get_tracking_uri())

# ✅ 2. Carregar modelo local salvo previamente via MLflow
modelo_final = mlflow.sklearn.load_model("data/06_models/modelo_vencedor")
print("[✔️] Modelo carregado localmente com sucesso")

# ✅ 3. Logar modelo no MLflow com artefatos
with mlflow.start_run(run_name="FixModeloComArtefato"):
    log_model(
        sk_model=modelo_final,
        artifact_path="modelo_vencedor",
        registered_model_name="ModeloArremessoKobe"
    )

    model_uri = mlflow.get_artifact_uri("modelo_vencedor")
    run_id = mlflow.active_run().info.run_id

    print("\n✔️ Modelo logado com sucesso no run:", run_id)
    print("📁 URI do artefato:", model_uri)

# ✅ 4. Registrar e aplicar alias `prod`
result = mlflow.register_model(model_uri, "ModeloArremessoKobe")
print(f"[📍] Nova versao registrada: v{result.version}")

client = MlflowClient()
client.set_registered_model_alias("ModeloArremessoKobe", "prod", result.version)
print(f"🏷️ Alias 'prod' aplicado na versao v{result.version}")

# ✅ 5. Carregar modelo via alias @prod
modelo_prod = mlflow.sklearn.load_model("models:/ModeloArremessoKobe@prod")
print("\n✅ Modelo carregado com sucesso via @prod")

# ✅ 6. Aplicar na base de producao
input_path = "../data/01_raw/dataset_kobe_prod.parquet"
df = pd.read_parquet(input_path)
colunas = ["lat", "lon", "minutes_remaining", "period", "playoffs", "shot_distance"]
df = df[colunas + (["shot_made_flag"] if "shot_made_flag" in df.columns else [])]
df = df.dropna(subset=colunas)
print(f"[INFO] Base de producao carregada e limpa: {df.shape}")

# 🔮 Fazer previsoes
y_pred = modelo_prod.predict(df[colunas])
y_proba = modelo_prod.predict_proba(df[colunas])[:, 1]
df_resultado = df.copy()
df_resultado["score"] = y_proba
df_resultado["shot_made_flag_predito"] = y_pred

# 📎 Salvar predicoes e logar como artefato
output_path = "../data/07_model_output/predicoes_prod.parquet"
Path(output_path).parent.mkdir(parents=True, exist_ok=True)
df_resultado.to_parquet(output_path, index=False)
mlflow.log_artifact(output_path)
print("[📎] Predicoes salvas em:", output_path)

# 📊 Avaliar se a base de producao tem y_true
if "shot_made_flag" in df.columns:
    y_true = df["shot_made_flag"].dropna()  # ✅ Remove NaNs para evitar erro
    y_pred_filtered = y_pred[df["shot_made_flag"].notna()]
    y_proba_filtered = y_proba[df["shot_made_flag"].notna()]

    f1 = f1_score(y_true, y_pred_filtered)
    loss = log_loss(y_true, y_proba_filtered)

    mlflow.log_metric("f1_score_prod", f1)
    mlflow.log_metric("log_loss_prod", loss)
    print(f"[📊] Avaliacao - F1: {f1:.4f} | LogLoss: {loss:.4f}")
else:
    print("[⚠️] Variavel alvo ausente - metricas nao calculadas")


[INFO] Tracking URI: file:///C:/Users/Cristina/Documents/myproject/mlruns
[✔️] Modelo carregado localmente com sucesso


Registered model 'ModeloArremessoKobe' already exists. Creating a new version of this model...
Created version '6' of model 'ModeloArremessoKobe'.



✔️ Modelo logado com sucesso no run: 26fa6618a91c49e69347fd6c96e73a91
📁 URI do artefato: file:///C:/Users/Cristina/Documents/myproject/mlruns/0/26fa6618a91c49e69347fd6c96e73a91/artifacts/modelo_vencedor
[📍] Nova versao registrada: v7
🏷️ Alias 'prod' aplicado na versao v7


Registered model 'ModeloArremessoKobe' already exists. Creating a new version of this model...
Created version '7' of model 'ModeloArremessoKobe'.



✅ Modelo carregado com sucesso via @prod
[INFO] Base de producao carregada e limpa: (6426, 7)
[📎] Predicoes salvas em: ../data/07_model_output/predicoes_prod.parquet
[📊] Avaliacao - F1: 0.0000 | LogLoss: 0.6392
